In [2]:
import requests
from bs4 import BeautifulSoup

username = "mariagps"
user_password = "8Z4vn*!WTP"
role = "editor"
login_data = {"username": username, "password": user_password,  "role": "editor",
              "hdnManuscriptServicesDisplayed" : "false",   'hdnNeedMultipLoginDropdown' : 0,
              'hdnOrcidIsAuthenticated' : 'False', 'hdnOrcidIsDuplicateEmail' : 'False',
              'hdnSsoLoginEnabled' : 'False', 'hdnUseOrcideLogin' : 'True', 'ignoreWarning' : 0,
              'initiateAscoLogin' : 'False', 'orcidAuthenticated' : 0, 'orcidLogin' : 0
 }
#login_data = {"username": username, "password": user_password, "Login": "submit", "role": "editor"}
headers = {
'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
'Accept-Encoding': 'gzip, deflate, br',
'Accept': '*/*',
'Connection': 'keep-alive',
'sec-ch-ua' : '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
'sec-ch-ua-mobile' : '?0',
'sec-ch-ua-platform' : '"Windows"',
'DNT' : '1',
'Upgrade-Insecure-Requests' : '1',
'Content-Type' : 'application/x-www-form-urlencoded',
'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36',

}

## login 

url = "https://www.editorialmanager.com/nefro/LoginAction.ashx" #este funciona

session = requests.Session()
response = session.get(url)
print(response.status_code)
#soup = BeautifulSoup(response.content, features="html.parser")
#print(soup)
states = ["__RequestVerificationToken", "Email", "RememberMe"]


post_request = session.post(url,  data=login_data, headers=headers, allow_redirects=True)

#print cookies
cookies = session.cookies.get_dict()
print(cookies)


200
{'EMSessionID': '%7B11C825A4%2DAA80%2D41B8%2DA311%2D73DD15C32AF3%7D'}


In [190]:
#save html
#with open("login.html", "w") as f:
    #f.write(post_request.text)


No veo el html de la página, parece sólo js

In [191]:
#call the main page and save html
url = "https://www.editorialmanager.com/nefro/default2.aspx"
response = session.get(url)
with open("main.html", "w") as f:
    f.write(response.text)

Aquí parece que entro en la página, no me muestra la cabecera pero no hay un link de login, así que asumo que estoy logueado. Pruebo a meterme en Menú ppal

In [192]:
#call manin menu and save html
url = 'https://www.editorialmanager.com/nefro/manuscript_status.asp'
response = session.get(url)
with open("menu.html", "w") as f:
    f.write(response.text)

Efectivamente, entro en el Menú ppal de autor

Pruebo a hacer logout y a imprimir la página

In [193]:
'''url_logout = 'https://www.editorialmanager.com/nefro/logout.asp'
#call the page
r = requests.post(url_logout, data=login_data, cookies=cookies, allow_redirects=True)
print(r.status_code)
#save the html
with open('logout.html', 'w') as f:
    f.write(r.text)'''

"url_logout = 'https://www.editorialmanager.com/nefro/logout.asp'\n#call the page\nr = requests.post(url_logout, data=login_data, cookies=cookies, allow_redirects=True)\nprint(r.status_code)\n#save the html\nwith open('logout.html', 'w') as f:\n    f.write(r.text)"

perfect. Si imprimo ahora main menu me lleva a inicio de sesión. :)

Ya sé hacer login y logout de la web pero en la página de autor. Tengo que cambiar a editor.

In [3]:
#url = "https://www.editorialmanager.com/nefro/Default.aspx?pg=EditorsMainMenu.aspx"
#url = "https://www.editorialmanager.com/nefro/default2.aspx"
url = "https://www.editorialmanager.com/nefro/manuscript_status.asp"
data = {'role': 'editor',}
 #   '__EVENTTARGET': 'RoleDropdown',
#add defaultMenu to cookies: defaultMenu=0; Path=/nefro/; Domain=editorialmanager.com; Secure;
cookies['defaultMenu'] = '0'
cookies['CurrentLang'] = 'es-ES'
cookies['Path'] = '/nefro/'
cookies['Domain'] = 'editorialmanager.com'


headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    #add the cookie from the variable cookies
    'Cookie': 'CurrentLang=es-ES; EMSessionID='+cookies['EMSessionID'],
    #'Cookie': 'CurrentLang=es-ES; EMSessionID=%7B72CDCD21%2D8534%2D4800%2D9B32%2D719C1E125623%7D'
}

response = session.post(url, data=data, headers=headers, cookies=cookies, allow_redirects=True)
print(response.status_code)

with open("Role_editor.html", "w") as f:
    f.write(response.text)



200


In [4]:
#manuscritos asignados
url = 'https://www.editorialmanager.com/nefro/AllAssignedSubmissions.aspx'

response = session.post(url, data=data, headers=headers, cookies=cookies, allow_redirects=True)
print(response.status_code)

with open("manuscritos_asignados.html", "w") as f:
    f.write(response.text)


200


No lo veo en el navegador, pero parece que sí que me he traído el html de manuscritos asignados. Intento scrapearlo.

Lo que necesito está dentro de un iframe #document ==$0

In [5]:
#Using regular expressions find all the occurrences between the tags <td class=\'  \' style=\'vertical-align: middle;;\'> and </td>
import re
#extract from body
with open("manuscritos_asignados.html", "r") as f:
    body = f.read()
    #print(body)
    #find all the occurrences between the tags <td class=\'  \' style=\'vertical-align: middle;;\'> and </td>
    #pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>.*?</td>')
    pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>.*?</td>')
    matches = pattern.findall(body)
    #print(matches)


In [13]:
#print (matches[0])

In [95]:
#create a dataframe called df_Asignados with the columns: numero_manuscrito, tipo_articulo, titulo, autor, fecha_inicial, fecha_estado, estado_actual, editor, estado_evaluacion, decision_editor
import pandas as pd
df_asignados = pd.DataFrame(columns=['numero_manuscrito', 'tipo_articulo', 'titulo', 'autor', 'fecha_inicial', 
                'fecha_estado', 'estado_actual', 'editor', 'estado_evaluacion', 'decision_editor','doc_id', 'Completo', 'Aceptado', 'Invitado – Sin Respuesta', 'Rechazado', 'Tarde'])

#the columns 10-15 are integers
df_asignados[['doc_id', 'Completo', 'Aceptado', 'Invitado – Sin Respuesta', 'Rechazado', 'Tarde']] = df_asignados[['doc_id', 'Completo', 'Aceptado', 'Invitado – Sin Respuesta', 'Rechazado', 'Tarde']].astype(int)

In [96]:
df_asignados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   numero_manuscrito         0 non-null      object
 1   tipo_articulo             0 non-null      object
 2   titulo                    0 non-null      object
 3   autor                     0 non-null      object
 4   fecha_inicial             0 non-null      object
 5   fecha_estado              0 non-null      object
 6   estado_actual             0 non-null      object
 7   editor                    0 non-null      object
 8   estado_evaluacion         0 non-null      object
 9   decision_editor           0 non-null      object
 10  doc_id                    0 non-null      int64 
 11  Completo                  0 non-null      int64 
 12  Aceptado                  0 non-null      int64 
 13  Invitado – Sin Respuesta  0 non-null      int64 
 14  Rechazado                 0 non-null      i

In [97]:
#function that find all the rs-headerRow elements
def find_estado_evaluacion(estado_ev_test, id):
    estado_evaluacion_elemento = []
    soup = BeautifulSoup(estado_ev_test, 'html.parser')
    #print('soup',soup.prettify())
    
    #rs_headerRow = soup.find_all('div', class_="rs-headerRow")
    #print('rs_headerRow', rs_headerRow)
    #find the dta-docid from <div class='reviewerHoverDetails leftAligned' data-docid='15961'>
    docid = soup.find('div', class_="reviewerHoverDetails leftAligned")['data-docid']
    #print('doc id', docid)
    #append the docid to the list estado_evaluacion_elemento
    estado_evaluacion_elemento.append('docid')
    estado_evaluacion_elemento.append(docid)

    #add th id to the dataframe
    df_asignados.loc[id, 'doc_id'] = docid


    rows = soup.select('.reviewerHoverDetails .rs-headerRow')
    for row in rows:
        count = row.select_one('.rs-countCell span').text
        detail = row.select_one('.rs-detailCell span').text
        #print(count, detail)
        #append the detail and count to the list estado_evaluacion_elemento
        estado_evaluacion_elemento.append(detail)
        estado_evaluacion_elemento.append(count)
        # add the count to the dataframe in the column detail
        df_asignados.loc[id, detail] = count


    return



In [98]:
#extract the values from the matches
for i in range(len(matches)):
    pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>')
    values = pattern.split(matches[i])
    #remove the first element because it is empty
    values.pop(0)
    #print(values[1])
    for j in range (len(values)):
        #print(j, values[j])
        #Titulo and autor are in the same field. If the value is the titulo, split in two and save in the corresponding column
        if j == 2:
            #split in two the string by <td class='  ' style='vertical-align: middle;text-align: center;'> 
            pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;text-align: center;\'>')
            title_author = pattern.split(values[j])
            #print(title_author)
            #store in the corresponding column
            df_asignados.loc[i, 'titulo'] = title_author[0]
            df_asignados.loc[i, 'autor'] = title_author[1]

        else:
            #if the column is >2, add 1 to the column number
            if j > 2:
                df_asignados.loc[i, df_asignados.columns[j+1]] = values[j]
            else:

                df_asignados.loc[i, df_asignados.columns[j]] = values[j]

    #expand estado_evaluacion to columns: doc_id, Completo, Aceptado, Invitado, Rechazado, Tarde
    estado_evaluacion = df_asignados.loc[i, 'estado_evaluacion']
    #print(estado_evaluacion)
    if estado_evaluacion != '':
        find_estado_evaluacion(estado_evaluacion, i)
    
    

#df_asignados.head()


In [99]:
df_asignados

,numero_manuscrito,tipo_articulo,titulo,autor,fecha_inicial,fecha_estado,estado_actual,editor,estado_evaluacion,decision_editor,doc_id,Completo,Aceptado,Invitado – Sin Respuesta,Rechazado,Tarde
0,NEFRO-D-23-00011,Original,"RELATIONSHIP BETWEEN U-MMP7 AND PROGNOSIS, COM...",Fadime Altinbas,11/01/2023,11/01/2023,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
1,NEFRO-D-23-00010,Original,The Association of Vitamin D Supplementation i...,"Ilhan Kilic, M.D.",09/01/2023,10/01/2023,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
2,NEFRO-D-23-00009,Notas técnicas,Actualización en el tratamiento con fluidos de...,"Alejandro Gonzalez-Castro, Ph.D, M.D.",09/01/2023,10/01/2023,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
3,NEFRO-D-23-00008,Original,Correlation of Luminex-based single-antigen-ba...,"Suleyman Rustu Rustu Oguz, MD PhD",06/01/2023,09/01/2023,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
4,NEFRO-D-23-00006,Original,Therapeutic adherence in kidney transplanted a...,Ana Barbosa Rodrigues,04/01/2023,05/01/2023,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
5,NEFRO-D-23-00005,Revisiones cortas,ANEMIA RENAL: TRATAMIENTOS ACTUALES Y MOLÉCULA...,"MANUEL HERAS BENITO, M.D",04/01/2023,05/01/2023,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
6,NEFRO-D-23-00002,Original,Sobrevida y su relación con el tipo de transfe...,"Rafael Alberto Gómez Acevedo, M.D.",31/12/2022,10/01/2023,Pendiente del Editor,Miguel Perez Fontan,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
7,NEFRO-D-22-00457,Cartas al editor,Dying with end stage kidney disease: factors a...,"Ana Cunha Rodrigues, M.D.",30/12/2022,31/12/2022,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
8,NEFRO-D-22-00455,Original,El desarrollo de una lesión renal aguda es un ...,"Borja Quiroga, MD PhD",28/12/2022,28/12/2022,Pendiente del Editor,Mariano Rodriguez,,<td class='colresize-cell ' style=';'></td>,NaN,NaN,NaN,NaN,NaN,NaN
9,NEFRO-D-22-00454,Original,Activación y control de enfermedad de paciente...,Alicia Cana,23/12/2022,01/01/2023,Pendiente Evaluadores,Mariano Rodriguez,<div class='reviewerHoverDetails leftAligned' ...,<td class='colresize-cell ' style=';'></td>,16058,1,1,1,1,NaN


# Errores:
-  [ok] hay que extraer el estado de la evaluación
* Hay que extraer la decisión del editor

Amplío las columnas para añadir los estados de la evaluación

In [49]:
#print estado_evaluacion of the row 9
id = 9
estado_evaluacion_test = (df_asignados.loc[id, 'estado_evaluacion'])
if estado_evaluacion_test != '':
    print(find_estado_evaluacion(estado_evaluacion_test, id))
else:
    print('None')

['docid', '16058', 'Completo', '1', 'Aceptado', '1', 'Invitado – Sin Respuesta', '1', 'Rechazado', '1']


In [25]:
#dictionary with the status and the corresponding number
#the status could be doc_id, Completo, Aceptado, Invitado, Rechazado, Tarde

status = {'docid':0,'Completo': 0, 'Aceptado': 0, 'Invitado': 0, 'Rechazado': 0, 'Tarde': 0}

In [26]:
#find count and detail of the estado_evaluacion

estado_evaluacion_elemento = find_estado_evaluacion(estado_evaluacion_test)
print(estado_evaluacion_elemento)

#fill the dictionary status with the values. Check every key and find the corresponding value in the list estado_evaluacion_elemento
for key in status:
    #print(key)
    for i in range(len(estado_evaluacion_elemento)):
        #print(estado_evaluacion_elemento[i])
        if key == estado_evaluacion_elemento[i]:
            status[key] = estado_evaluacion_elemento[i+1]
            break


print(status)



NameError: name 'find_estado_evaluacion' is not defined

In [215]:
rs_headerRow

[<div class="rs-headerRow rs-complete"><div class="rs-countCell"><span class="rs-overlay-cell">1</span></div><div class="rs-detailCell"><span class="rs-overlay-cell">Completo</span></div></div>]

In [115]:
#extract the content from estado_evaluacion and stre in a list

#go through the dataframe and extract the content from estado_evaluacion and store in a list
estado_evaluacion = []
for k in range(len(df_asignados)):
    element = df_asignados.loc[k, 'estado_evaluacion']
    #print(element)

    #check if the element is not empty
    if element != '':
        count, detail = find_estado_evaluacion(element)
        estado_evaluacion.append(count)
        

'''        #print the numero_manuscrito
        print(df_asignados.loc[k, 'numero_manuscrito'])
        
        soup = BeautifulSoup(element, 'html.parser')

        #find the div class="ro-content"
        div = soup.find('div', class_="ro-content")
        #print(div)'''

    else:
        estado_evaluacion.append('')
        #print('elemento ',k, 'vacío')

print(estado_evaluacion)





elemento  9 1 Completo
elemento  9 1 Aceptado
elemento  9 1 Invitado – Sin Respuesta
elemento  9 1 Rechazado

elemento  9 1 Completo
elemento  9 1 Aceptado
elemento  9 1 Invitado – Sin Respuesta
elemento  9 1 Rechazado

elemento  9 1 Completo
elemento  9 1 Aceptado
elemento  9 1 Invitado – Sin Respuesta
elemento  9 1 Rechazado

elemento  9 1 Completo
elemento  9 1 Aceptado
elemento  9 1 Invitado – Sin Respuesta
elemento  9 1 Rechazado


elemento  11 1 Aceptado
elemento  11 2 Invitado – Sin Respuesta

elemento  11 1 Aceptado
elemento  11 2 Invitado – Sin Respuesta


elemento  13 1 Completo
elemento  13 2 Invitado – Sin Respuesta
elemento  13 1 Rechazado

elemento  13 1 Completo
elemento  13 2 Invitado – Sin Respuesta
elemento  13 1 Rechazado

elemento  13 1 Completo
elemento  13 2 Invitado – Sin Respuesta
elemento  13 1 Rechazado


elemento  14 1 Completo
elemento  14 2 Aceptado
elemento  14 1 Tarde

elemento  14 1 Completo
elemento  14 2 Aceptado
elemento  14 1 Tarde

elemento  14 1 

In [69]:
#split between the tags <td class='  ' style='vertical-align: middle;;'> and <

pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>.*?</td>')
matches = pattern.findall(body)

print (matches[20])



<td class='  ' style='vertical-align: middle;;'>NEFRO-D-22-00288R1<td class='  ' style='vertical-align: middle;;'>Original<td class='  ' style='vertical-align: middle;;'>Dysregulation of ferroptosis may participate in the mitigating effect of CoCl2 on contrast-induced nephropathy<td class='  ' style='vertical-align: middle;text-align: center;'>Huilin Li <td class='  ' style='vertical-align: middle;;'>04/08/2022<td class='  ' style='vertical-align: middle;;'>09/12/2022<td class='  ' style='vertical-align: middle;;'>Pendiente Evaluadores<td class='  ' style='vertical-align: middle;;'>Mariano Rodriguez<td class='  ' style='vertical-align: middle;;'><div class='reviewerHoverDetails leftAligned' data-docid='15804'><div class=rs-table><div class='rs-headerRow rs-complete'><div class='rs-countCell '><span class='rs-overlay-cell'>1</span></div><div class='rs-detailCell'><span class='rs-overlay-cell'>Completo</span></div></div><div class='rs-headerRow rs-agreed'><div class='rs-countCell '><span

In [132]:
#splitwith the tag <td class='  ' style='vertical-align: middle;;'> 
pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>')
#values = pattern.split(matches[0])
#remove the first element
#print (values)
print (matches[3])
#split the value 2 in 2 parts by <td class='  ' style='vertical-align: middle;text-align: center;'> and the end of the string
pattern2 = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;text-align: center;\'>')
values2 = pattern2.split(matches[0][3])
print(values)



<td class='  ' style='vertical-align: middle;;'>NEFRO-D-23-00008<td class='  ' style='vertical-align: middle;;'>Original<td class='  ' style='vertical-align: middle;;'>Correlation of Luminex-based single-antigen-based results with CDC-XM and FC-XM results<td class='  ' style='vertical-align: middle;text-align: center;'>Suleyman Rustu Rustu Oguz, MD PhD <td class='  ' style='vertical-align: middle;;'>06/01/2023<td class='  ' style='vertical-align: middle;;'>09/01/2023<td class='  ' style='vertical-align: middle;;'>Pendiente del Editor<td class='  ' style='vertical-align: middle;;'>Mariano Rodriguez<td class='  ' style='vertical-align: middle;;'><td class='  ' style='vertical-align: middle;;'><td class='colresize-cell  ' style=';'></td>
['', 'NEFRO-D-23-00011', 'Original', "RELATIONSHIP BETWEEN U-MMP7 AND PROGNOSIS, COMORBIDITY AND MORTALITY IN AKI<td class='  ' style='vertical-align: middle;text-align: center;'>Fadime Altinbas ", '11/01/2023', '11/01/2023', 'Pendiente del Editor', 'Mari

['', 'NEFRO-D-23-00010', 'Original', "The Association of Vitamin D Supplementation in case of need on Chronic Kidney Disease Progression<td class='  ' style='vertical-align: middle;text-align: center;'>Ilhan Kilic, M.D. ", '09/01/2023', '10/01/2023', 'Pendiente del Editor', 'Mariano Rodriguez', '', "<td class='colresize-cell  ' style=';'></td>"]

In [97]:
#print the column estado_evaluacion data fo the row 0
estado_ev_test =df_asignados.loc[9, 'estado_evaluacion']
print(estado_ev_test)

<div class='reviewerHoverDetails leftAligned' data-docid='16058'><div class=rs-table><div class='rs-headerRow rs-complete'><div class='rs-countCell '><span class='rs-overlay-cell'>1</span></div><div class='rs-detailCell'><span class='rs-overlay-cell'>Completo</span></div></div><div class='rs-headerRow rs-agreed'><div class='rs-countCell '><span class='rs-overlay-cell'>1</span></div><div class='rs-detailCell'><span class='rs-overlay-cell'>Aceptado</span></div></div><div class='rs-headerRow rs-invited'><div class='rs-countCell '><span class='rs-overlay-cell'>1</span></div><div class='rs-detailCell'><span class='rs-overlay-cell'>Invitado &#8211; Sin Respuesta</span></div></div><div class='rs-headerRow rs-declined'><div class='rs-countCell '><span class='rs-overlay-cell'>1</span></div><div class='rs-detailCell'><span class='rs-overlay-cell'>Rechazado</span></div></div><div class=\"rs-more-link \"><span class=\"rs-more-link linkActive\" >(más...)</span></div></div>


In [105]:
#scrap the data from estado_evaluacion div class="ro-content" with beautiful soup

soup = BeautifulSoup(estado_ev_test, 'html.parser')
#print(soup.prettify())

#find the div class="ro-content"
div = soup.find('div', class_="ro-content")
#print(div)

#find all the rs-headerRow elements
rs_headerRow = soup.find_all('div', class_="rs-headerRow")
for i in range(len(rs_headerRow)):
    #print(rs_headerRow[i])
    print('')
    rows = soup.select('.reviewerHoverDetails .rs-headerRow')
    for row in rows:
        count = row.select_one('.rs-countCell span').text
        detail = row.select_one('.rs-detailCell span').text
        print(count, detail)










1 Completo
1 Aceptado
1 Invitado – Sin Respuesta
1 Rechazado

1 Completo
1 Aceptado
1 Invitado – Sin Respuesta
1 Rechazado

1 Completo
1 Aceptado
1 Invitado – Sin Respuesta
1 Rechazado

1 Completo
1 Aceptado
1 Invitado – Sin Respuesta
1 Rechazado


In [91]:
#extract from div





1 Completo
